In [7]:

import numpy as np
import math
import cv2

capture=cv2.VideoCapture(0)
color=(255,255,0)
thickness=5
capture.set(3,2000)
capture.set(4,2000)
points=[]
canvas=np.zeros((720, 1280, 3),np.uint8)


while capture.isOpened():
    
    ret,frame=capture.read()
    frame=cv2.flip(frame, 1)
    
    green= cv2.rectangle(frame, (0,0), (100,100), (0,255,0), -1) 
    blue= cv2.rectangle(frame, (0,150), (100,250), (255,0,0), -1) 
    red= cv2.rectangle(frame, (0,300), (100,400), (0,0,255), -1)
    yellow= cv2.rectangle(frame, (0,450), (100,550), (0,255,255), -1)
    black= cv2.rectangle(frame, (0,600), (100,700), (0,0,0), -1)

    cv2.putText(frame,'Eraser', (0, 650),  cv2.FONT_HERSHEY_SIMPLEX , 1,  (100, 255,10),  1,  cv2.LINE_4) 

    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_blue = np.array([127, 107, 45])
    upper_blue = np.array([163, 223, 155])
    mask=cv2.inRange(hsv,lower_blue,upper_blue)
    mask = cv2.erode(mask,np.ones((5,5)),iterations = 1)
    mask = cv2.dilate(mask,np.ones((5,5)),iterations = 2)
    contours, _ =cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    
    x=0
    y=0
    count=0
    
    for contour in contours:
        approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        area=cv2.contourArea(contour)
        if count==0:
            points.append((0,0))
            count=count+1
        if((len(approx)>8)&(area>100)):
            c = max(contours, key = cv2.contourArea)
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
            height=int((x+x+w)/2)
            width=int((y+h+y)/2)
            points.append((height,width))
            points.append((height+6, width+2))

            if((x>0 and x<100) and (y>0 and y<100)):
                color=(0,255,0)
                thickness=5
            elif((x>0 and x<100) and (y<250 and y>150)):
                color= (255,0,0)
                thickness=5
            elif((x>0 and x<100) and (y<400 and y>300)):
                color=(0,0,255)
                thickness=5
            elif((x>0 and x<100) and (y>450 and y<550)):
                color=(0,255,255)
                thickness=5
            elif((x>0 and x<100) and (y<700 and y>600)):
                color=(0,0,0)
                thickness=50
            cv2.line(canvas,points[-1],points[-2],color,thickness)
    
    frame = cv2.add(frame,canvas)
    cv2.imshow("frame",frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()